This Jupyter notebook file is for the final Coursera Capstone project as part of the Applied Data Science specialization by IBM. 

As a final assignment, the goal is to come up with an idea to use the provided accident data to predict the various accidents' severity. Important factors are the project validity and relevance towards the community or a group of people. 

For this the **CR**oss **I**ndustry **S**tandard **P**rocess for **D**ata **M**ining (CRISP-DM) approach is used. With CRISP-DM the following 6 steps are used:
1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment

The first 5 steps will be shown below, the last step "6. Deployment" is this documentation.


In [5]:
# Before beginning, the following libraries are imported and the CSV data file is loaded

import numpy as np
import pandas as pd

source = 'https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv'
df_raw = pd.read_csv(source, index_col = False)

print('Data-Collisions.csv loaded')


Data-Collisions.csv loaded


1. Business Understanding

What is the application perspective to the project's objective

Mainly text (WEEK 1 GOAL)

In [ ]:
# Script for 1. Bus.Und. if required

2. Data Understanding

What are usefull attributes/colums for modeling (WEEK 1 GOAL)

In [ ]:
# Script for 2. Data Und. if required

3. Data Preparation

All requirements to create the final dataset for modeling



In [6]:
# All required script for prep work of 3. Data Prep
df_raw.set_index('OBJECTID', inplace=True) # The ObjectID column can be set as index colum
df_raw.index.name = None # Remove the index name, happy df is happy
df_raw.head()

print('Happy df is happy!')

,SEVERITYCODE,X,Y,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
1,2,-122.323148,47.703140,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
2,1,-122.347294,47.647172,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
3,1,-122.334540,47.607871,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
4,1,-122.334803,47.604803,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
5,2,-122.306426,47.545739,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


4. Modeling

Build the model, in this case supervised machine learing (see paper)


In [3]:
# All required script for 4. Modeling

5. Evaluation

Check the results if they allow for 1. Bus.Und. to be achieved. Think of R2 rules
Make sure that all is clear why it works or not.

In [4]:
# All script for 5. Evaluation